In [1]:
import pandas as pd

In [2]:
# read data
combas = pd.read_csv('../data/FS_Combas.csv')
comins = pd.read_csv('../data/FS_Comins.csv')
cnmont = pd.read_csv('../data/TRD_Cnmont.csv')
mnth = pd.read_csv('../data/TRD_Mnth.csv')
nrrate = pd.read_csv('../data/TRD_Nrrate.csv')

In [3]:
# rename time cols
combas.rename(columns={'Accper': 'Time'}, inplace=True)
comins.rename(columns={'Accper': 'Time'}, inplace=True)
cnmont.rename(columns={'Trdmnt': 'Time'}, inplace=True)
mnth.rename(columns={'Trdmnt': 'Time'}, inplace=True)
nrrate.rename(columns={'Clsdt': 'Time'}, inplace=True)

In [4]:
# convert time, drop duplicates, drop nan
for df in [combas, comins, cnmont, mnth, nrrate]:
    df['Time'] = pd.to_datetime(df['Time'])
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)

In [5]:
# Typrep = 'A' needed
combas = combas[combas['Typrep'] == 'A']
comins = comins[comins['Typrep'] == 'A']
# '06-30', '12-31' needed
combas = combas[combas['Time'].dt.month % 6 == 0]
comins = comins[comins['Time'].dt.month % 6 == 0]
# merge combas and comins 
finance = pd.merge(combas, comins)
finance.to_csv('../data/step1/finance.csv')

In [6]:
cnmont = cnmont[cnmont['Markettype'] == 5]
mnth = mnth[(mnth['Markettype'] == 1) | (mnth['Markettype'] == 4)]
cnmont = cnmont[['Time', 'Cmretwdos']]
mnth = mnth[['Stkcd', 'Time', 'Msmvosd', 'Mretwd']]
nrrate = nrrate[['Time', 'Nrrmtdt']]
returns = pd.merge(cnmont, mnth, how='right', on='Time')
returns = pd.merge(returns, nrrate, how='left', on='Time')
returns = returns[['Stkcd', 'Time', 'Cmretwdos', 'Msmvosd', 'Mretwd', 'Nrrmtdt']]
returns['Msmvosd'] = returns['Msmvosd'] * 1000
returns.to_csv('../data/step1/returns.csv')